In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

2024-11-30 14:05:32.567582: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
file_path = "ibm.csv"
file_cleaned_path = "cleaned_ibm.csv"
data = pd.read_csv(file_path)

data_filtered = data[["Date", "Open", "High", "Low", "Volume", "Close"]].sort_values(by="Date")
data_cleaned = data_filtered.dropna()

data_cleaned.to_csv(file_cleaned_path, index=False)
data = pd.read_csv(file_cleaned_path)
data.head()

,Date,Open,High,Low,Volume,Close
0,1962-01-02 00:00:00-05:00,1.518550,1.518550,1.501487,407940,1.501487
1,1962-01-03 00:00:00-05:00,1.501487,1.514612,1.501487,305955,1.514612
2,1962-01-04 00:00:00-05:00,1.514613,1.514613,1.498863,274575,1.499519
3,1962-01-05 00:00:00-05:00,1.497551,1.497551,1.467363,384405,1.469988
4,1962-01-08 00:00:00-05:00,1.468675,1.468675,1.430613,572685,1.442425


In [3]:
data["Target"] = data["Close"].shift(-1)
data.head()

,Date,Open,High,Low,Volume,Close,Target
0,1962-01-02 00:00:00-05:00,1.518550,1.518550,1.501487,407940,1.501487,1.514612
1,1962-01-03 00:00:00-05:00,1.501487,1.514612,1.501487,305955,1.514612,1.499519
2,1962-01-04 00:00:00-05:00,1.514613,1.514613,1.498863,274575,1.499519,1.469988
3,1962-01-05 00:00:00-05:00,1.497551,1.497551,1.467363,384405,1.469988,1.442425
4,1962-01-08 00:00:00-05:00,1.468675,1.468675,1.430613,572685,1.442425,1.459488


In [4]:
final_data_row = data.tail(1)
data.drop(data.tail(1).index, inplace=True)
data.dropna(inplace=True)
final_data_row

,Date,Open,High,Low,Volume,Close,Target
15825,2024-11-13 00:00:00-05:00,209.5,211.410004,209.070099,2186158,210.669998,NaN


In [5]:
data.iloc[:, 1:].corr()['Target']

Open      0.999714
High      0.999765
Low       0.999767
Volume    0.133558
Close     0.999812
Target    1.000000
Name: Target, dtype: float64

In [6]:
# Features scaling
model_features = data.drop("Target", axis=1).drop("Date", axis=1)
model_target = data["Target"]

model_feature_scaler = MinMaxScaler()
model_feature_scaler.fit(model_features)
model_scaled_features = pd.DataFrame(model_feature_scaler.transform(model_features), columns=model_features.columns.tolist())

model_target_scaler = MinMaxScaler()
model_target_scaler.fit(model_target.values.reshape(-1,1))
model_scaled_target = pd.DataFrame(model_target_scaler.transform(model_target.values.reshape(-1,1)), columns=["Target"])

In [7]:
def create_dataset(X_data, y_data, t_steps):
    X, y = [], []
    for i in range(len(X_data) - t_steps):
        v = X_data.iloc[i:(i + t_steps)].values
        X.append(v)
        y.append(y_data.iloc[i + t_steps])
    return np.array(X), np.array(y)

In [8]:
t_steps = 60
X, y = create_dataset(model_scaled_features, model_scaled_target, t_steps)
X.shape, y.shape

((15765, 60, 5), (15765, 1))

In [9]:
train_size = int(0.80 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((12612, 60, 5), (3153, 60, 5), (12612, 1), (3153, 1))

In [10]:
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

2024-11-30 14:05:54.602257: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2027] TensorFlow was not built with CUDA kernel binaries compatible with compute capability 8.0. CUDA kernels will be jit-compiled from PTX, which could take 30 minutes or longer.
2024-11-30 14:05:54.607783: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2027] TensorFlow was not built with CUDA kernel binaries compatible with compute capability 8.0. CUDA kernels will be jit-compiled from PTX, which could take 30 minutes or longer.
2024-11-30 14:05:55.678641: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 541 MB memory:  -> device: 0, name: NVIDIA A30, pci bus id: 0000:17:00.0, compute capability: 8.0


(TensorShape([12612, 60, 5]),
 TensorShape([12612, 1]),
 TensorShape([3153, 60, 5]),
 TensorShape([3153, 1]))

In [11]:
input_shape = (X_train.shape[1], X_train.shape[2])  # (time_step, features)
input_shape

(60, 5)

In [12]:
# GRU model
num_epochs = 30
batch_size = 64

gru_model = Sequential([
    tf.keras.layers.GRU(units=50, return_sequences=True, input_shape=input_shape),
    tf.keras.layers.GRU(units=50),
    Dense(units=1)
])

In [13]:
gru_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 60, 50)            8550      
                                                                 
 gru_1 (GRU)                 (None, 50)                15300     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 23,901
Trainable params: 23,901
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compiling the GRU model
gru_model.compile(optimizer='adam', loss='mean_squared_error')

# Training the GRU model
history_gru = gru_model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size, validation_data=(X_test, y_test))

Epoch 1/30


2024-11-30 14:06:14.454067: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:433] Could not create cudnn handle: CUDNN_STATUS_NOT_INITIALIZED
2024-11-30 14:06:14.454188: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Possibly insufficient driver version: 550.54.15
2024-11-30 14:06:14.454212: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at cudnn_rnn_ops.cc:1554 : UNKNOWN: Fail to find the dnn implementation.


UnknownError: Graph execution error:

Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
	 [[sequential/gru/PartitionedCall]] [Op:__inference_train_function_5784]

In [ ]:
# Make predictions and evaluate GRU model
gru_predictions = gru_model.predict(X_test)

# Add dummy columns to match the original scaled data shape (5 columns)
dummy_columns = np.zeros((gru_predictions.shape[0], 4))  # 4 dummy columns to match Open, High, Low, Volume
predicted_prices_scaled = np.concatenate((dummy_columns, gru_predictions), axis=1)

# Apply inverse transformation to get back to original scale
predicted_prices = model_feature_scaler.inverse_transform(predicted_prices_scaled)[:, -1]  # Get only the Close column

# Similarly for actual prices
actual_prices_scaled = np.concatenate((dummy_columns, y_test.numpy().reshape(-1, 1)), axis=1)
actual_prices = model_feature_scaler.inverse_transform(actual_prices_scaled)[:, -1]

gru_mse, gru_rmse, gru_mae = calculate_metrics(actual_prices, predicted_prices)
print(f'GRU Model - MSE: {gru_mse:.4f}, RMSE: {gru_rmse:.4f}, MAE: {gru_mae:.4f}')

In [ ]:
plt.plot(history_gru.history['loss'], label='Training Loss')
plt.plot(history_gru.history['val_loss'], label='Validation Loss')
plt.title(' GRU Training vs. Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# plotting code with log scale
plt.plot(history_gru.history['loss'], label='Training Loss')
plt.plot(history_gru.history['val_loss'], label='Validation Loss')
plt.title('GRU Training vs. Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.yscale('log')  # Setting the y-axis to logarithmic scale

plt.legend()
plt.show()

### Predicted vs actual price curve

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(actual_prices, color="blue", label="Actual IBM Stock Price")
plt.plot(predicted_prices, color="red", label="Predicted IBM Stock Price by GRU")
plt.title("IBM Stock Price Predictions - GRU ")
plt.xlabel("Dates")
plt.ylabel("Stock Price")
plt.legend()
plt.show()

### Random search for hyperparameter optimization

In [ ]:
import random
import numpy as np
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


param_ranges = {
    "units": (32, 256), 
    "batch_size": (16, 128),
    "learning_rate": (1e-5, 1e-2),
    "time_steps": (30, 120) }

n_samples = 20

def generate_random_params():
    units = random.randint(param_ranges["units"][0], param_ranges["units"][1])
    batch_size = random.randint(param_ranges["batch_size"][0], param_ranges["batch_size"][1])
    learning_rate = random.uniform(param_ranges["learning_rate"][0], param_ranges["learning_rate"][1])
    time_steps = random.randint(param_ranges["time_steps"][0], param_ranges["time_steps"][1])
    return units, batch_size, learning_rate, time_steps

def evaluate_model(units, batch_size, learning_rate, time_steps):

    global X_train, y_train, X_test, y_test
    X, y = create_dataset(model_scaled_features, model_scaled_target, time_steps)
    train_size = int(0.80 * len(X))
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]


    X_train_tensor = tf.convert_to_tensor(X_train, dtype=tf.float32)
    y_train_tensor = tf.convert_to_tensor(y_train, dtype=tf.float32)
    X_test_tensor = tf.convert_to_tensor(X_test, dtype=tf.float32)
    y_test_tensor = tf.convert_to_tensor(y_test, dtype=tf.float32)
    
    input_shape = (time_steps, X_train.shape[2])

    gru_model = Sequential([
        tf.keras.layers.GRU(units=units, return_sequences=True, input_shape=input_shape,
        tf.keras.layers.GRU(units=units),
        Dense(units=1)
    ])

    gru_model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=0)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=0)


    # Training model
    history = gru_model.fit(
        X_train_tensor, y_train_tensor,
        epochs=10,
        batch_size=batch_size,
        validation_data=(X_test_tensor, y_test_tensor),
        callbacks=[early_stopping, reduce_lr],
        verbose=1
    )

    val_loss = min(history.history['val_loss'])
    return val_loss

# Performing random search

# Number of random samples 
n_samples= 20 
best_params = None
best_loss = float("inf")

for i in range(n_samples):
    units, batch_size, learning_rate, time_steps = generate_random_params()

    # Log the parameters being tested
    print(f"\nTesting Hyperparameters: Units = {units}, Batch Size = {batch_size}, "
          f"Learning Rate = {learning_rate:.6f}, Time Steps = {time_steps}")
    
    # Evaluate the model
    val_loss = evaluate_model(units, batch_size, learning_rate, time_steps)
    print(f"X_train shape: {X_train.shape}, X_test shape: {X_test.shape}")


    # Log the result
    print(f"Validation Loss: {val_loss:.6f}")

    # Update best parameters if the current loss is lower
    if val_loss < best_loss:
        best_loss = val_loss
        best_params = (units, batch_size, learning_rate, time_steps)

# Print the best hyperparameters and their validation loss
print(f"Best Parameters: Units = {best_params[0]} Batch Size = {best_params[1]}, "
      f"Learning Rate = {best_params[2]:.6f}, Time Steps (t_steps) = {best_params[3]}")
print(f"Best Validation Loss: {best_loss:.6f}")

## Now using these parameters to test GRU model again

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

t_steps = 107
X, y = create_dataset(model_scaled_features, model_scaled_target, t_steps)

train_size = int(0.80 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

#converting into tensor 
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)

input_shape = (X_train.shape[1], X_train.shape[2])

#batch size and no of epochs
num_epochs = 30
batch_size = 24

model_optimized = Sequential([
        tf.keras.layers.GRU(units=units, return_sequences=True, input_shape=input_shape,
        tf.keras.layers.GRU(units=units),
        Dense(units=1)
    ])
model_optimized.compile(optimizer=Adam(learning_rate=0.003813), loss='mean_squared_error')

# early_stopping = EarlyStopping(
#     monitor='val_loss',
#     patience=10,
#     restore_best_weights=True
# )

# # Reducing learning rate when plateau is reached
# reduce_lr = ReduceLROnPlateau(
#     monitor='val_loss',
#     factor=0.2,
#     patience=5,
#     min_lr=1e-5
# )

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=0)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=0)


history_gru = model_optimized.fit(
        X_train, y_train,
        epochs=num_epochs,
        batch_size=batch_size,
        validation_data=(X_test, y_test),
        callbacks=[early_stopping, reduce_lr],
        verbose=1
    )

## Again plotting the graphs based on new set of optimized hyperparameters

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler


def calculate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    return mse, rmse, mae

predictions = model_optimized.predict(X_test)

# Adding dummy columns to match the original scaled data shape (5 columns)
# 4 dummy columns to match Open, High, Low, Volume
dummy_columns = np.zeros((predictions.shape[0], 4))
predicted_prices_scaled = np.concatenate((dummy_columns, predictions), axis=1)

# Applying inverse transformation to get back to original scale
predicted_prices = model_feature_scaler.inverse_transform(predicted_prices_scaled)[:, -1]

# for actual prices
actual_prices_scaled = np.concatenate((dummy_columns, y_test.numpy().reshape(-1, 1)), axis=1)
actual_prices = model_feature_scaler.inverse_transform(actual_prices_scaled)[:, -1]

# Calculating metrics
lstm_mse, lstm_rmse, lstm_mae = calculate_metrics(actual_prices, predicted_prices)
print(f'GRU Model - MSE: {lstm_mse:.4f}, RMSE: {lstm_rmse:.4f}, MAE: {lstm_mae:.4f}')

In [ ]:
%matplotlib inline
plt.plot(history_rnn.history['loss'], label='Training Loss')
plt.plot(history_rnn.history['val_loss'], label='Validation Loss')
plt.title(' RNN Training vs. Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
%matplotlib inline
# Your existing plotting code with log scale
plt.plot(history_gru.history['loss'], label='Training Loss')
plt.plot(history_gru.history['val_loss'], label='Validation Loss')
plt.title('LSTM Training vs. Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.yscale('log')  # Setting the y-axis to logarithmic scale

plt.legend()
plt.show()

In [ ]:
# predicted and actual price plot 

plt.figure(figsize=(15, 10))
plt.plot(actual_prices, color="blue", label="Actual IBM Stock Price")
plt.plot(predicted_prices, color="red", label="Predicted IBM Stock Price by RNN")
plt.title("IBM Stock Price Predictions - RNN ")
plt.xlabel("Dates")
plt.ylabel("Stock Price")
plt.legend()
plt.show()

### Using PSO for optimizing hyperparameter for GRU 

In [ ]:
import random
import numpy as np
import pyswarm
from pyswarm import pso
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

param_ranges = {
    "units": (32, 256),       # Range for LSTM units
    "batch_size": (16, 128),       # Range for batch size
    "learning_rate": (1e-5, 1e-2), # Range for learning rate
    "t_steps": (30, 120)           # Range for sequence length
}

bounds = [
    param_ranges["units"],       # Bounds for LSTM units
    param_ranges["batch_size"],       # Bounds for batch size
    param_ranges["learning_rate"],    # Bounds for learning rate
    param_ranges["t_steps"]           # Bounds for sequence length
]

def objective_function(params):
    units = int(params[0])  
    batch_size = int(params[1])
    learning_rate = float(params[2])
    t_steps = int(params[3])

    print(f"\nTrying Hyperparameters: Units = {units}, Batch Size = {batch_size}, "
          f"Learning Rate = {learning_rate:.6f}, Sequence Length (t_steps) = {t_steps}")

    # Updating sequence length for data creation
    global X_train, y_train, X_test, y_test
    X, y = create_dataset(model_scaled_features, model_scaled_target, t_steps)
    train_size = int(0.80 * len(X))
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # Converting to tensors
    X_train_tensor = tf.convert_to_tensor(X_train, dtype=tf.float32)
    y_train_tensor = tf.convert_to_tensor(y_train, dtype=tf.float32)
    X_test_tensor = tf.convert_to_tensor(X_test, dtype=tf.float32)
    y_test_tensor = tf.convert_to_tensor(y_test, dtype=tf.float32)
    
    input_shape = (t_steps, X_train.shape[2])

    # Defining the model architecture
    model = Sequential([
        tf.keras.layers.GRU(units=units, return_sequences=True, input_shape=input_shape,
        tf.keras.layers.GRU(units=units),
        Dense(units=1)
    ])
    
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=0)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=0)

    # Training the model
    history = model.fit(
        X_train_tensor, y_train_tensor,
        epochs=10,
        batch_size=batch_size,
        validation_data=(X_test_tensor, y_test_tensor),
        callbacks=[early_stopping, reduce_lr],
        verbose=1
    )

    # validation loss
    val_loss = min(history.history['val_loss'])

    print(f"Validation Loss: {val_loss:.6f}")

    return val_loss

# Running PSO 
best_params, best_loss = pso(
    objective_function,
    lb=[b[0] for b in bounds],  # Lower bounds
    ub=[b[1] for b in bounds],  # Upper bounds
    swarmsize=5,               # Number of particles
    maxiter=5,                 # Number of iterations
    debug=True                  # Enable logging
)

print(f"\nBest Parameters: Units = {int(best_params[0])}, Batch Size = {int(best_params[1])}, "
      f"Learning Rate = {best_params[2]:.6f}, Sequence Length (t_steps) = {int(best_params[3])}")
print(f"Best Validation Loss: {best_loss:.6f}")

### Now using the hyperparameters from PSO 

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

t_steps = 107
X, y = create_dataset(model_scaled_features, model_scaled_target, t_steps)

train_size = int(0.80 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

#converting into tensor 
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)

input_shape = (X_train.shape[1], X_train.shape[2])

#batch size and no of epochs
num_epochs = 30
batch_size = 24

model_optimized = Sequential([
        tf.keras.layers.GRU(units=units, return_sequences=True, input_shape=input_shape,
        tf.keras.layers.GRU(units=units),
        Dense(units=1)
    ])
model_optimized.compile(optimizer=Adam(learning_rate=0.003813), loss='mean_squared_error')

# early_stopping = EarlyStopping(
#     monitor='val_loss',
#     patience=10,
#     restore_best_weights=True
# )

# # Reducing learning rate when plateau is reached
# reduce_lr = ReduceLROnPlateau(
#     monitor='val_loss',
#     factor=0.2,
#     patience=5,
#     min_lr=1e-5
# )

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=0)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=0)


history_gru = model_optimized.fit(
        X_train, y_train,
        epochs=num_epochs,
        batch_size=batch_size,
        validation_data=(X_test, y_test),
        callbacks=[early_stopping, reduce_lr],
        verbose=1
    )

## Again plotting the graphs based on new set of optimized hyperparameters from PSO

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler


def calculate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    return mse, rmse, mae

predictions = model_optimized.predict(X_test)

# Adding dummy columns to match the original scaled data shape (5 columns)
# 4 dummy columns to match Open, High, Low, Volume
dummy_columns = np.zeros((predictions.shape[0], 4))
predicted_prices_scaled = np.concatenate((dummy_columns, predictions), axis=1)

# Applying inverse transformation to get back to original scale
predicted_prices = model_feature_scaler.inverse_transform(predicted_prices_scaled)[:, -1]

# for actual prices
actual_prices_scaled = np.concatenate((dummy_columns, y_test.numpy().reshape(-1, 1)), axis=1)
actual_prices = model_feature_scaler.inverse_transform(actual_prices_scaled)[:, -1]

# Calculating metrics
lstm_mse, lstm_rmse, lstm_mae = calculate_metrics(actual_prices, predicted_prices)
print(f'GRU Model - MSE: {lstm_mse:.4f}, RMSE: {lstm_rmse:.4f}, MAE: {lstm_mae:.4f}')

In [ ]:
#training vs validation loss plot

%matplotlib inline
plt.plot(history_gru.history['loss'], label='Training Loss')
plt.plot(history_gru.history['val_loss'], label='Validation Loss')
plt.title(' RNN Training vs. Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
#training vs validation loss plot based on log scale 

%matplotlib inline
# Your existing plotting code with log scale
plt.plot(history_gru.history['loss'], label='Training Loss')
plt.plot(history_gru.history['val_loss'], label='Validation Loss')
plt.title('LSTM Training vs. Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.yscale('log')  # Setting the y-axis to logarithmic scale

plt.legend()
plt.show()

In [ ]:
# predicted and actual price plot 

plt.figure(figsize=(15, 10))
plt.plot(actual_prices, color="blue", label="Actual IBM Stock Price")
plt.plot(predicted_prices, color="red", label="Predicted IBM Stock Price by RNN")
plt.title("IBM Stock Price Predictions - RNN ")
plt.xlabel("Dates")
plt.ylabel("Stock Price")
plt.legend()
plt.show()